<a href="https://colab.research.google.com/github/Text-Analysis-Android-Apps/scripts/blob/main/ConseguirModelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este cuaderno lo tengo para convertir el modelo de T5 (en formato .h5) a TFLite, para usarlo desde Android

Primero consultamos por curiosidad la versión de Python instalada:

In [ ]:
!python --version

Python 3.7.13


Después, procedemos a iniciar la conversión.
Primero instalamos las dependencias:

In [ ]:
!pip install transformers
!pip install sentencepiece


     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 30.7 MB/s 
     |████████████████████████████████| 596 kB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 27.1 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.4 MB/s 


Este fue mi primer intento. Da fallo porque usamos el T5ForConditionalGeneration en vez de sin el TF delante. Y eso hace que no se pueda convertir:

In [ ]:
#from git import Repo
import tensorflow as tf
from transformers import TFDistilBertForQuestionAnswering, T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

#Repo.clone_from(git_url, repo_dir)

#Mio

#tokenizer = T5Tokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
#model = T5ForConditionalGeneration.from_pretrained("Vamsi/T5_Paraphrase_Paws")
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")


#model = TFDistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

input_spec = tf.TensorSpec([1, 384], tf.int32)
#model._set_inputs(input_spec, training=False)

#print(model.inputs)
#print(model.outputs)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# For normal conversion:
#converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

# For conversion with FP16 quantization:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True

# For conversion with hybrid quantization:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# converter.experimental_new_converter = True

tflite_model = converter.convert()

open("Vamsi_T5_Paraphrase_Paws.tflite", "wb").write(tflite_model)



AttributeError: ignored

Este fue mi segundo e infructuoso intento. Traté de cargar el modelo directamente como modelo de Keras sin especificar el tipo exacto de modelo que era. Para ello primero lo cloné en la carpeta t5:

In [ ]:
!git clone https://huggingface.co/Vamsi/T5_Paraphrase_Paws t5
!apt install git-lfs -y
!cd t5
!pwd
!cd t5 && git lfs install
!cd t5 && git lfs pull
!cd ..
!pwd

Cloning into 't5'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 36 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 3s (828 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


Y, finalmente, traté de cargarlo como un modelo Keras genérico:

In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model("t5/tf_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

ValueError: ignored

Otro intento más: https://github.com/Helsinki-NLP/Tatoeba-Challenge/issues/14 y https://www.tensorflow.org/lite/guide/ops_select#known_limitations para opsets incompatibles y https://starriet.medium.com/tensorflow-2-0-wanna-limit-gpu-memory-10ad474e2528 para RAM

En este último y exitoso intento, primero clono de nuevo el modelo en la carpeta t5 inicializando los archivos pesados con git lfs y, finalmente, creo la carpeta models para guardar el modelon final y cargo el modelo con su correspondiente clase: TFT5ForConditionalGeneration.
Esta clase la he sacado del archivo config.json del modelo. Los links de arriba tiene información de cómo saqué esta solución y de cómo arreglé algunos errores que me dio al principio. Lo malo es que el modelo producido ocupa más de 200 MB, y necesito menos de 200 MB para usarlo en Android Studio. Este modelo ocupa unos 213 MB.

In [ ]:
!git clone https://huggingface.co/Vamsi/T5_Paraphrase_Paws t5
!apt install git-lfs -y
!cd t5 && git lfs install
!cd t5 && git lfs pull


Cloning into 't5'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 36 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,575 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ..

In [ ]:
!mkdir models
import tensorflow as tf
from transformers import TFMarianMTModel, MarianTokenizer, AutoTokenizer, TFT5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM

#RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

model_name = './t5'
model = TFT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)

# Revisar si input_spec está bien
input_spec = {
    'decoder_input_ids': tf.TensorSpec(shape=(1, 20), dtype=tf.int32, name='decoder_input_ids'),
    'attention_mask': tf.TensorSpec(shape=(1, 20), dtype=tf.bool, name='attention_mask'),
    'input_ids': tf.TensorSpec(shape=(1, 20), dtype=tf.int32, name='input_ids')
}

model._saved_model_inputs_spec = None
model._set_save_spec(input_spec)

converter = tf.lite.TFLiteConverter.from_keras_model(model);
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert();

with open("./models/tf_model.tflite", 'wb') as o_:
    o_.write(tflite_model)

mkdir: cannot create directory ‘models’: File exists


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['lm_head.weight', 'encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


INFO:tensorflow:Assets written to: /tmp/tmp7y4ll42q/assets


INFO:tensorflow:Assets written to: /tmp/tmp7y4ll42q/assets


Para descargar el modelo tflite:

In [ ]:
from google.colab import files
import pandas as pd
files.download('./models/tf_model.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls ./models/

tf_model.tflite


Ahora vamos a intentar generar un modelo comprimido siguiendo las instrucciones de https://www.tensorflow.org/lite/performance/post_training_quantization . Este modelo ocupa unos 425 MB:

In [ ]:
!mkdir models
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration

#RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

model_name = './t5'
model = TFT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)

input_spec = {
    'decoder_input_ids': tf.TensorSpec(shape=(1, 20), dtype=tf.int32, name='decoder_input_ids'),
    'attention_mask': tf.TensorSpec(shape=(1, 20), dtype=tf.bool, name='attention_mask'),
    'input_ids': tf.TensorSpec(shape=(1, 20), dtype=tf.int32, name='input_ids')
}

model._saved_model_inputs_spec = None
model._set_save_spec(input_spec)

converter = tf.lite.TFLiteConverter.from_keras_model(model);
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#Optimización de tamaño:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert();

with open("./models/tf_model_quantified.tflite", 'wb') as o_:
    o_.write(tflite_model)

mkdir: cannot create directory ‘models’: File exists


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_head.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


INFO:tensorflow:Assets written to: /tmp/tmp7y3wt54x/assets


INFO:tensorflow:Assets written to: /tmp/tmp7y3wt54x/assets


Para descargar el modelo cuantificado:

In [ ]:
from google.colab import files
import pandas as pd
files.download('./models/tf_model_quantified.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Vamos a intentar descargar el vocabulario que usa este modelo para crear los tokens de entrada:

In [ ]:
import json
import tensorflow as tf
from transformers import T5Tokenizer
import sentencepiece as spm

#RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

model_name = './t5'
#model = TFT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)
tokenizer = T5Tokenizer.from_pretrained(model_name, from_pt=True)
#tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")

!echo "Antes:"
!ls -l
!echo "Fin antes"
tokenizer.save_vocabulary(save_directory='.')
sp = spm.SentencePieceProcessor()
print("Spiece: "+str(sp.Load('t5/spiece.model')))
vocabs = [sp.IdToPiece(id) for id in range(sp.GetPieceSize())]
with open("./vocab.txt", 'w') as o_:
  for v in vocabs:
    o_.write(v+'\n')
!echo "Después:"
!ls -l
#with open( 'word_dict.json' , 'w' ) as file:
#    json.dump( tokenizer.word_index , file )

ModuleNotFoundError: ignored

Para descargar el vocabulario:

In [ ]:
from google.colab import files
import pandas as pd
files.download('./vocab.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Vamos a usar la línea de comandos de sentencepiece: https://github.com/google/sentencepiece/blob/master/README.md:

In [ ]:
#Instalar la versión de C++ de sentencepiece:
!sudo apt-get install cmake build-essential pkg-config libgoogle-perftools-dev
! git clone https://github.com/google/sentencepiece.git 
! cd sentencepiece && mkdir build && cd build
! cd sentencepiece && cd build && cmake ..
! cd sentencepiece && cd build && make -j $(nproc)
! cd sentencepiece && cd build && sudo make install
! cd sentencepiece && cd build && sudo ldconfig -v

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libunwind-dev
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 627 kB of archives.
After this operation, 6,761 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libunwind-dev amd64 1.2.1-8 [423 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgoogle-perftools-dev amd64 2.5-2.2ubuntu3 [204 kB]
Fetched 627 kB in 0s (2,094 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/

In [ ]:
!spm_export_vocab --model=t5/spiece.model --output=vocab2.txt
from google.colab import files
files.download('./vocab2.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ahora vamos a probar a reconvertir el modelo a ver si sale mejor con un JSON: https://stackoverflow.com/questions/63598159/how-to-convert-just-a-h5-file-to-a-tflite-file/63619212#63619212:


In [ ]:
!ls t5/

import sys, os
import tensorflow as tf
import traceback

from os.path                    import splitext, basename

print(tf.__version__)

mod_path = "t5/tf_model.h5"

def load_model(path,custom_objects={},verbose=0):
    #from tf.keras.models import model_from_json

    path = splitext(path)[0]
    with open('t5/config.json','r') as json_file:
        model_json = json_file.read()
    model = tf.keras.models.model_from_json(model_json, custom_objects=custom_objects)
    model.load_weights('%s.h5' % path)
    if verbose: print('Loaded from %s' % path)
    return model

keras_mod = load_model(mod_path)

converter = tf.lite.TFLiteConverter.from_keras_model(keras_mod)
tflite_model = converter.convert()

# Save the TF Lite model.
with open("./models/tf_model_JSON.tflite", 'wb') as o_:
    o_.write(tflite_model)
#with tf.io.gfile.GFile('model.tflite', 'wb') as f:
    #f.write(tflite_model)

'checkpointepoch=0.ckpt'   pytorch_model.bin	     spiece.model
 config.json		   README.md		     tf_model.h5
 flax_model.msgpack	   special_tokens_map.json   tokenizer_config.json
2.8.0


ValueError: ignored

Probamos a conseguir el tflite como el primer intento pero cambiando los input spec (3 min - 216 MB):

In [ ]:
!mkdir models
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration
from google.colab import files

#RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

#model_name = './t5'
#model = TFT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)
model = TFT5ForConditionalGeneration.from_pretrained('Vamsi/T5_Paraphrase_Paws')

# Revisar si input_spec está bien
#input_spec = tf.TensorSpec([1, 256], tf.int32)
input_spec = {
    'decoder_input_ids': tf.TensorSpec([1, 256], tf.int32),
    'input_ids': tf.TensorSpec([1, 256], tf.int32)
}
#input_spec = {
#    'decoder_input_ids': tf.TensorSpec(shape=(1, 20), dtype=tf.int32, name='decoder_input_ids'),
#    'attention_mask': tf.TensorSpec(shape=(1, 20), dtype=tf.bool, name='attention_mask'),
#    'input_ids': tf.TensorSpec(shape=(1, 20), dtype=tf.int32, name='input_ids')
#}

model._saved_model_inputs_spec = None
model._set_save_spec(input_spec)

converter = tf.lite.TFLiteConverter.from_keras_model(model);
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert();

with open("./models/tf_model_v2.tflite", 'wb') as o_:
    o_.write(tflite_model)

files.download('./models/tf_model_v2.tflite')

mkdir: cannot create directory ‘models’: File exists


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_head.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


INFO:tensorflow:Assets written to: /tmp/tmpnq0oxary/assets


INFO:tensorflow:Assets written to: /tmp/tmpnq0oxary/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Con Pytorch:

In [ ]:
!git clone https://github.com/omerferhatt/torch2tflite
!cd torch2tflite && python setup.py install

Cloning into 'torch2tflite'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 11), reused 22 (delta 4), pack-reused 0
Unpacking objects: 100% (39/39), done.
running install
running bdist_egg
running egg_info
creating torch2tflite.egg-info
writing torch2tflite.egg-info/PKG-INFO
writing dependency_links to torch2tflite.egg-info/dependency_links.txt
writing requirements to torch2tflite.egg-info/requires.txt
writing top-level names to torch2tflite.egg-info/top_level.txt
writing manifest file 'torch2tflite.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'torch2tflite.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/torch2tflite
copying torch2tflite/__init__.py -> build/lib/torch2tflite
copying torch2tflite/converter.py -> build/lib/torch2t

In [ ]:
!ls t5/pytorch_model.bin
!python3 -m torch2tflite.converter --torch-path t5/pytorch_model.bin --tflite-path pytorch2tflite.tflite --sample-file sample_image.png

t5/pytorch_model.bin
ERROR:root:Specified file path not compatible with torch2tflite, exiting!


Sin definir salidas y entradas raras:

In [ ]:
!mkdir models
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration
from google.colab import files

#RAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    print(e)

model = TFT5ForConditionalGeneration.from_pretrained('Vamsi/T5_Paraphrase_Paws')

converter = tf.lite.TFLiteConverter.from_keras_model(model);
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert();

model_path = './models/tf_model_sin_entradas_ni_salidas_raras.tflite'

with open(model_path, 'wb') as o_:
    o_.write(tflite_model)

files.download(model_path)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at Vamsi/T5_Paraphrase_Paws.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


INFO:tensorflow:Assets written to: /tmp/tmpptpek6bu/assets


INFO:tensorflow:Assets written to: /tmp/tmpptpek6bu/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Hay que definir entradas y salidas. Voy a probar como en este GIST: https://gist.github.com/jhabr/aced0ee37d6a585616e3edce47706235:

In [ ]:
!mkdir models
from transformers import TFT5ForConditionalGeneration, T5Config
import tensorflow as tf
from google.colab import files

model_name = 'Vamsi/T5_Paraphrase_Paws'

# load pretrained model
config = T5Config.from_pretrained(model_name, num_labels=10) #CUIDADO! No sé por qué tienen que ser 10 labels
model = TFT5ForConditionalGeneration.from_pretrained(model_name, config=config)

# create input and output
SEQUENCE_LENGTH = 256
num_return_sequences = 5
NO_CLASSES = num_return_sequences #10 antes

input_ids = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), dtype=tf.int32, name="input_ids", batch_size=1)
attention_mask = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), dtype=tf.int32, name="attention_mask", batch_size=1)
inputs = [input_ids, attention_mask]
outputs = model(input_ids, attention_mask)[0]
outputs = tf.keras.layers.Dense(NO_CLASSES, activation='softmax')(outputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

# export model to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
converter.allow_custom_ops = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# For conversion with FP16 quantization
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_model = converter.convert()

model_path = './models/tf_model_como_distilbert.tflite'

with open(model_path, 'wb') as o_:
    o_.write(tflite_model)

files.download(model_path)

mkdir: cannot create directory ‘models’: File exists


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at Vamsi/T5_Paraphrase_Paws.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


ValueError: ignored

Basándome en este otro ejemplo: https://github.com/legacyai/tf-transformers/blob/main/docs/source/tflite_tutorials/roberta_tflite.ipynb:

In [ ]:
!mkdir models
from transformers import TFT5ForConditionalGeneration, T5Config
import tensorflow as tf
from google.colab import files

model_name = 'Vamsi/T5_Paraphrase_Paws'

SEQUENCE_LENGTH = 256
num_return_sequences = 5
NO_CLASSES = num_return_sequences #10 antes
batch_size = 1


# load pretrained model
config = T5Config.from_pretrained(model_name, num_labels=num_return_sequences) #CUIDADO! No sé por qué tienen que ser 10 labels
model = TFT5ForConditionalGeneration.from_pretrained(model_name, config=config, batch_size=batch_size, sequence_length=SEQUENCE_LENGTH)

# export model to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
converter.allow_custom_ops = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# For conversion with FP16 quantization
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_model = converter.convert()

model_path = './models/tf_model_como_distilbert.tflite'

with open(model_path, 'wb') as o_:
    o_.write(tflite_model)

files.download(model_path)

mkdir: cannot create directory ‘models’: File exists


TypeError: ignored